In [ ]:
import json
import psqlgraph
import os

from graphviz import Digraph
from IPython.display import SVG, display
from gdcdatamodel.models import *
from gdcdatamodel.models.versioned_nodes import VersionedNode
from psqlgraph import *
from sqlalchemy import *

HOST = os.environ.get('PG_HOST')
USER = os.environ.get('PG_USER')
DATABASE = os.environ.get('PG_DATABASE')
PASSWORD = os.environ.get('PG_PASSWORD')

# PG_USER= PG_HOST= PG_DATABASE= PG_PASSWORD=   jupyter notebook


def visualize(nodes):
    dot = Digraph()
    dot.graph_attr['rankdir'] = 'RL'

    edges_added = set()
    nodes = {node.node_id: node for node in nodes}
    
    print('Graphing {} nodes.'.format(len(nodes)))

    for node in nodes.values():

        label = '{}({})'.format(node.__class__.__name__, node.node_id[:8])
        dot.node(node.node_id, label)

        for edge in node.edges_out:
            if  edge not in edges_added and edge.dst_id in nodes:
                dot.edge(edge.src_id, edge.dst_id)
                edges_added.add(edge)
                
        for edge in node.edges_in:
            if edge not in edges_added and edge.src_id in nodes:
                dot.edge(edge.src_id, edge.dst_id)
                edges_added.add(edge)

    display(dot)
    
    print(json.dumps({str(node): node.props for node in nodes.values()}, indent=2))

g = psqlgraph.PsqlGraphDriver(HOST, USER, PASSWORD, DATABASE)
print('Ready!')

In [ ]:
with g.session_scope():
    cases = g.nodes(Case).subq_path('samples').limit(1).all()
    case_neighbors = [edge.src for case in cases for edge in case.edges_in]
    samples = [sample for case in cases for sample in case.samples]
    portions = [portion for sample in samples for portion in sample.portions]
    analytes = [analyte for portion in portions for analyte in portion.analytes]
    aliquots = [aliquot for analyte in analytes for aliquot in analyte.aliquots]
    nodes = cases + case_neighbors + samples + portions + analytes + aliquots
    visualize(nodes)